# Title

## Overview

## Business Problem

Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

Movie production has significant up-front costs that will require internal stakeholder support to adequately fund new projects. Further, it will be important to generate engagement with Microsoft's titles both to maximize return on investment and to legitimize Microsoft as a content producer in the future. Therefore, this analysis aims to generate recommendations on how best to deploy the content production budget. 

We will use data from imdb, TheNumbers and XXX to determine answers to the following questions:
- What genres of movie are likely to optimize return on investment?

- What is the relationship between movie budget and popularity? - TMDB Popularity score

- What season/month should we target releases in order to optimize our return on investment?





## Data Understanding

For this analysis, we will utilize a database from IMDB and a dataset from TheNumbers.com. The database contains robust information for each title--most notably the genre, release date and IMDB popularity score. The dataset from TheNumbers.com will primarily be used for return on investment data, including worldwide gross revenue and movie production budget. The Movie Database contains popularity score.

In [ ]:
# import necessary packages
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import zipfile
import numpy as np
%matplotlib inline

The IMDB dataset includes information on movies spanning nearly a century. Given the age of many of th

In [ ]:
# Extract IMDb SQL .db file
with zipfile.ZipFile('./Data/im.db.zip') as zipObj:
    # Extract all contents of .zip file into current directory
    zipObj.extractall(path='./Data/')
    
# Create connection to IMDb DB
conn = sqlite3.connect('./data/im.db')

## Data Preparation

### Data Cleaning

### Merging Datasets

### Feature Engineering

## Analysis

### Return on Investment by Genre

### Popularity by Investment

### Popularity by Budget

## Conclusions


### Next Steps